In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from dynamic_model.dataset import DynamicsModelDataset
from dynamic_model.model import DynamicsModel
from dynamic_model.train import train_model




/home/hamza/PycharmProjects/StateCompression/dynamic_model/model.py:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if layer_idx is 0:
/home/hamza/PycharmProjects/StateCompression/dynamic_model/model.py:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if layer_idx is 0:


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(f"Using Device: {device}")
torch.backends.cudnn.benchmark = True

Using Device: cuda:0


## Load Dataset

In [3]:

def load_dataset(file_path, input_key, output_key, test_size= 0.3, device=device):

    dataset = np.load(file_path, allow_pickle=True)
    x = dataset[()][input_key]
    y = dataset[()][output_key]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)
    train_dataset = DynamicsModelDataset(x_train,y_train, device)
    validation_dataset = DynamicsModelDataset(x_test,y_test, device)
    return train_dataset, validation_dataset


dataset_input_key = 'merged_input'
dataset_output_key = 'merged_output'
dataset_file_path = 'tmp/ib-out/ib-samples.npy'




In [4]:

batch_size = 8000
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 0,
          'drop_last':True}



train_dataset, val_dataset = load_dataset(file_path=dataset_file_path, input_key=dataset_input_key, output_key= dataset_output_key)
train_loader = DataLoader(train_dataset, **params)
val_loader = DataLoader(val_dataset, **params)

In [5]:
num_of_features = train_dataset.get_features_size()
seq_len = train_dataset.get_seq_len()
hidden_size = 32
out_size = train_dataset.get_output_feature_size()

model = DynamicsModel(features=num_of_features,hidden_size=hidden_size,out_size=out_size, batch_size=batch_size, seq_len=seq_len, n_layers=2).to(device=device)

In [6]:
torch.autograd.set_detect_anomaly(True)
train_model(model=model, train_loader=train_loader, test_loader=val_loader, n_epochs=10000, learning_rate=1e-2)

Untrained test
--------
Test loss: 13143.432291666666

Epoch 0
---------
Train loss: 11857.842087518602
Test loss: 10541.911051432291
Epoch time: epoch_time = 12.726s
Epoch 1
---------
Train loss: 9772.029610770089
Test loss: 8883.919162326389
Epoch time: epoch_time = 12.632s
Epoch 2
---------
Train loss: 8412.58485049293
Test loss: 7795.26666937934
Epoch time: epoch_time = 12.620s
Epoch 3
---------
Train loss: 7461.519804454985
Test loss: 7016.9521077473955
Epoch time: epoch_time = 12.619s
Epoch 4
---------
Train loss: 6729.555181594122
Test loss: 6267.1862386067705
Epoch time: epoch_time = 12.614s
Epoch 5
---------
Train loss: 6141.75832984561
Test loss: 5791.808281792535
Epoch time: epoch_time = 12.591s
Epoch 6
---------
Train loss: 5702.405261811756
Test loss: 5433.032660590277
Epoch time: epoch_time = 12.695s
Epoch 7
---------
Train loss: 5383.560023716518
Test loss: 5150.893079969618
Epoch time: epoch_time = 12.811s
Epoch 8
---------
Train loss: 5140.65154157366
Test loss: 4945.5

/home/hamza/PycharmProjects/StateCompression/dynamic_model/model.py:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if layer_idx is 0:
/home/hamza/PycharmProjects/StateCompression/dynamic_model/model.py:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if layer_idx is 0:
/home/hamza/PycharmProjects/StateCompression/dynamic_model/model.py:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if layer_idx is 0:
/home/hamza/PycharmProjects/StateCompression/dynamic_model/model.py:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if layer_idx is 0:


KeyboardInterrupt: 